# Primeros pasos con la API de Gemini de google
Podes copiar el código de los diferentes bloques y ejecutarlos en un colab de google: https://colab.google/

## Ejemplo básico
En este ejemplo se muestra cómo usar la API de Gemini de Google para generar texto.
Se configura un cliente con una API key y se hace una consulta simple al modelo,
especificando parámetros como:
- El modelo de gemini a usar
- El prompt o consulta
- Configuraciones opcionales como tokens máximos, temperatura y una instrucción del sistema


In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="TU_API_KEY") # Agrega tu API key

instruccion_del_modelo = "Necesito que respondas mezclando palabras en inglés y español"

response = client.models.generate_content(
    model="gemini-2.0-flash-lite", # Elegir el modelo de gemini que quieras usar
    contents="Como son los perros?", # El prompt que quieras usar
    config=types.GenerateContentConfig(
        max_output_tokens=200, # Opcional: El número maximo de tokens para la respuesta
        temperature=0, # Opcional: El grado de aleatoriedad de la respuesta
        system_instruction=instruccion_del_modelo # Opcional: La instrucción que quieres que siga el modelo
    )
)

response.text

## Usar la API con lista de mensajes
Para usar la API con una lista de mensajes, necesitamos pasar una lista de diccionarios donde cada uno representa un mensaje del usuario o del modelo. Cada mensaje debe tener un "role" que puede ser "user" o "model", y una lista de "parts" que contiene el contenido del mensaje en formato texto.


In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key="TU_API_KEY") # Agrega tu API key

instruccion_del_modelo = "Siempre devolvé un ejemplo de código en tus respuestas"

messages = [
    {
        "role": "user",
        "parts": [
            {
              "text": "Cómo funcionan los diccionarios en python?"
            }
        ]
    },
    {
        "role": "model",
        "parts": [
            {
            "text": "Los diccionarios de python son parecidos a los objetos de javascript."
            }
        ]
    },
    {
        "role": "user",
        "parts": [
            {
            "text": "Bueno, esa respuesta no me dice mucho. Expayate un poco."
            }
        ]
    },
]

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=types.GenerateContentConfig(
        temperature=0,
        system_instruction=instruccion_del_modelo
    )
)

response.text

## Caso de uso: analista de contratos

Este ejemplo muestra cómo usar Gemini para analizar contratos y obtener respuestas estructuradas. En lugar de obtener texto libre que luego tendríamos que procesar manualmente, configuramos el modelo para que nos devuelva la información en un formato específico y estructurado.

### ¿Qué es un Structured Output?
Un structured output (salida estructurada) es una respuesta del modelo que sigue una estructura específica y predefinida, en lugar de texto libre. Esto es especialmente útil cuando:
- Necesitamos procesar la respuesta programáticamente
- Queremos asegurarnos que la respuesta tenga un formato consistente
- Necesitamos integrar la respuesta en otros sistemas o aplicaciones

### Definiendo el Schema con Pydantic
Pydantic es una biblioteca de Python que nos permite definir schemas o estructuras de datos con validación. En este ejemplo:

```python
class Clausula(BaseModel):
    nombre: str
    breve_descripcion: str

class ResumenDeContrato(BaseModel):
    breve_resumen: str
    clausulas: list[Clausula]
```

Este schema define que queremos que el modelo nos devuelva:
- Un resumen breve del contrato
- Una lista de cláusulas, donde cada cláusula tiene un nombre y una descripción breve

### Configuración para Structured Output
Para que Gemini devuelva una respuesta estructurada, necesitamos configurar dos parámetros especiales en la llamada a la API:
- `response_mime_type='application/json'`: Indica que queremos la respuesta en formato JSON
- `response_schema=ResumenDeContrato`: Define la estructura que debe seguir ese JSON

De esta manera, en lugar de texto libre, obtenemos un objeto JSON que podemos procesar fácilmente en nuestro código y que está garantizado que seguirá la estructura que definimos con Pydantic.

In [ ]:
from pydantic import BaseModel # Para definir el esquema o estructura de la respuesta
import json

from google import genai
from google.genai import types

client = genai.Client(api_key="TU_API_KEY")

instruccion_del_modelo = "Sos un experto en analizar contratos y ayudar a las personas a entenderlo."

contrato = """
CONTRATO DE LOCACIÓN

Entre el Sr. Juan Pérez, DNI N° 30.123.456, con domicilio en Av. Corrientes 1234, Ciudad Autónoma de Buenos Aires, en adelante denominado "LOCADOR", y el Sr. Matías Gómez, DNI N° 35.987.654, con domicilio en Av. Santa Fe 5678, Ciudad Autónoma de Buenos Aires, en adelante denominado "LOCATARIO", convienen en celebrar el presente CONTRATO DE LOCACIÓN, sujeto a las siguientes cláusulas:

PRIMERA - Objeto: El LOCADOR da en locación al LOCATARIO el inmueble ubicado en Calle Ficticia 123, Departamento 4B, Ciudad Autónoma de Buenos Aires, destinado exclusivamente para vivienda del LOCATARIO y su grupo familiar.

SEGUNDA - Plazo: La locación se establece por un plazo de 24 (veinticuatro) meses, contados a partir del 1 de abril de 2025 hasta el 31 de marzo de 2027.

TERCERA - Canon locativo: El LOCATARIO abonará al LOCADOR la suma mensual de $250.000 (pesos doscientos cincuenta mil), pagaderos por adelantado entre los días 1 y 5 de cada mes en la cuenta bancaria que designe el LOCADOR.

CUARTA - Ajustes: El valor del alquiler se ajustará cada seis meses conforme al índice establecido por el Banco Central de la República Argentina (ICL - Índice de Contratos de Locación).

QUINTA - Depósito en garantía: El LOCATARIO entrega en este acto la suma de $250.000 (pesos doscientos cincuenta mil) en concepto de depósito en garantía, que será restituido al finalizar el contrato, previa verificación del estado del inmueble y deducción de eventuales deudas o daños.

SEXTA - Destino y prohibiciones: El inmueble será destinado exclusivamente para uso residencial. Se prohíbe subalquilar, ceder o dar en préstamo total o parcialmente el inmueble sin consentimiento expreso del LOCADOR.

SÉPTIMA - Servicios y expensas: El LOCATARIO abonará los servicios de luz, gas, agua, internet y expensas ordinarias del inmueble, mientras que el LOCADOR será responsable de las expensas extraordinarias.

OCTAVA - Conservación del inmueble: El LOCATARIO se compromete a mantener el inmueble en buen estado de conservación y limpieza, realizando a su cargo las reparaciones menores por uso y desgaste normal. Las reparaciones estructurales correrán por cuenta del LOCADOR.

NOVENA - Resolución anticipada: El LOCATARIO podrá rescindir el contrato transcurridos seis meses desde el inicio, notificando con una anticipación de al menos 30 días y abonando la indemnización correspondiente conforme a la Ley de Alquileres vigente.

DÉCIMA - Garantía: El LOCATARIO presenta como garantía un seguro de caución emitido por la compañía Aseguradora Ficticia S.A., cuyo costo será abonado por el LOCATARIO durante toda la vigencia del contrato.

UNDÉCIMA - Entrega y restitución: Al finalizar el contrato, el LOCATARIO deberá restituir el inmueble en el mismo estado en que lo recibió, salvo deterioros derivados del uso normal.

DUODÉCIMA - Jurisdicción: Para cualquier cuestión derivada de este contrato, las partes se someten a la jurisdicción de los Tribunales Ordinarios de la Ciudad Autónoma de Buenos Aires.

En prueba de conformidad, se firman dos ejemplares de idéntico tenor y a un solo efecto, en la Ciudad Autónoma de Buenos Aires, a los 1 días del mes de marzo de 2025.

FIRMA LOCADOR: ___________________________

FIRMA LOCATARIO: ___________________________
"""

# Definimos el esquema o estructura de la respuesta
class Clausula(BaseModel):
  nombre: str
  breve_descripcion: str

class ResumenDeContrato(BaseModel):
  breve_resumen: str
  clausulas: list[Clausula]

# Definimos lo que vamos a enviar al modelo: el contrato y una instrucción para que el modelo haga un resumen
messages = [
    {
        "role": "user",
        "parts": [
            {
              "text": contrato
            },
            {
                "text": "Necesito info resumida de este contrato"
            }
        ]
    },
]


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=types.GenerateContentConfig(
        temperature=0,
        system_instruction=instruccion_del_modelo,
        response_mime_type='application/json', # Para que el modelo devuelva un JSON
        response_schema=ResumenDeContrato # Para que el modelo devuelva un JSON con la estructura que queremos
    ),
)

json.loads(response.text)

## Caso de uso: analisis de imagenes
En este caso de uso, vamos a ver cómo usar Gemini para analizar imágenes.
Gemini tiene la capacidad de procesar imágenes y responder preguntas sobre ellas.
Para esto necesitamos:
1. Importar las librerías necesarias (google.genai y PIL para procesar imágenes)
2. Tener una imagen guardada en una carpeta de nuestro sistema y definir su ruta
3. Cargar la imagen usando PIL
4. Enviar la imagen junto con una consulta al modelo
5. El modelo analizará la imagen y responderá la pregunta


In [ ]:
from google import genai
from google.genai import types
from PIL import Image # Para abrir y procesar imagenes


client = genai.Client(api_key="TU_API_KEY")


ruta_de_la_imagen="/content/test.png" # Definimos la ruta de la imagen que queremos procesar
imagen = Image.open(ruta_de_la_imagen) # Abrimos la imagen

instruccion_del_modelo = "Tu tarea es analizar las imagenes que te pasan los usarios y responder sus consultas"


response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[imagen, "de que color son los techos de esta imagen?"], # Definimos la imagen y la instrucción para el modelo
    config=types.GenerateContentConfig(
        temperature=0,
        system_instruction=instruccion_del_modelo
    )
)

response.text